In [29]:
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import pandas as pd
import copy

In [30]:
df = pd.read_csv('https://raw.githubusercontent.com/khongtrii/dataset/master/Data/heart_failure_clinical_records_dataset.csv')

In [63]:
df.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

In [31]:
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [39]:
len(df[df['DEATH_EVENT'] == 0]), len(df[df['DEATH_EVENT'] == 1])

(203, 96)

In [32]:
train, val, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8 *len(df))])

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [33]:
### Import thư viện
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import pandas as pd
import copy
def getX_y_scaler(dataframe, y_label, x_labels=None, oversample=False):
  dataframe = copy.deepcopy(dataframe)
  if x_labels is None:
    X = dataframe[[c for c in dataframe.columns if c != y_label]].values
  else:
    if len(x_labels)==1:
      X = dataframe[x_labels[0]].values.reshape(-1,1)
    else:
      X = dataframe[x_labels].values
  y = dataframe[y_label].values
  scalar = StandardScaler()
  X = scalar.fit_transform(X)
  if oversample:
    ros = RandomOverSampler()
    X, y = ros.fit_resample(X, y)


  data = np.hstack((X,np.reshape(y, (-1,1))))
  return data, X,y

train, X_train, y_train = getX_y_scaler(train, 'DEATH_EVENT', x_labels = df.columns[:-1],oversample=True)
val, X_val, y_val = getX_y_scaler(val, 'DEATH_EVENT', x_labels = df.columns[:-1],oversample=False)
test, X_test, y_test = getX_y_scaler(test, 'DEATH_EVENT', x_labels = df.columns[:-1],oversample=False)

In [34]:
from sklearn.svm import SVC
svc_model = SVC()
svc_model.fit(X_train, y_train)

SVC()

In [35]:
svc_model.score(X_test, y_test)

0.7166666666666667

In [104]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score

def score_model(model, X_score, y_score):
  print(model)
  print('Score: ', model.score(X_score, y_score))
  print("MSE: ", mean_squared_error(y_score, model.predict(X_score)))
  print("MAE: ", mean_absolute_error(y_score, model.predict(X_score)))
  print("R2: ", r2_score(y_score, model.predict(X_score)))
  print("Accuracy: ", accuracy_score(y_score, model.predict(X_score)))
  print("F1: ", f1_score(y_score, model.predict(X_score)))

score_model(svc_model, X_test, y_test)

SVC()
Score:  0.7166666666666667
MSE:  0.2833333333333333
MAE:  0.2833333333333333
R2:  -0.3953488372093028
Accuracy:  0.7166666666666667
F1:  0.5142857142857143


In [98]:
a = [1,2,3,4,5]
b = [6, 7, 8, 9, 10]
c = [11, 12, 13, 14, 15]
data = pd.DataFrame({'a' : a, 'b' : b, 'c' : c})

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       5 non-null      int64  
 1   b       4 non-null      float64
 2   c       3 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 248.0 bytes


In [93]:
from sklearn.impute import SimpleImputer
def solve_missing_values(dataframe):
  cols = []
  for col in dataframe.columns:
    missing_data = dataframe[col].isnull().sum()
    if missing_data > 0:
      cols.append(col)
  for i in cols:
    imputer = SimpleImputer(strategy='mean')
    dataframe[i] = imputer.fit_transform(dataframe[i].values.reshape(-1,1))
    dataframe[i] = round(dataframe[i], 3)
  data = dataframe
  return data

In [66]:
data.columns

Index(['a', 'b', 'c'], dtype='object')

In [99]:
data = solve_missing_values(data)

In [100]:
data

,a,b,c
0,1,6,11
1,2,7,12
2,3,8,13
3,4,9,14
4,5,10,15
